In [1]:
from PyQt6.QtWidgets import QFileDialog
from layout import ApplicationLayout
import pandas, numpy
from core.data import mapping, extract
from itertools import product
from collections import Counter
from core.optimizer import optimize
from core.data.schema import WeightMinMax, DataFrame
from core.data import transform

data = pandas.DataFrame({"weight": 1, "min": pandas.NA, "max": pandas.NA}, dtype="Int64", index=list(mapping.runes.effects.values()))
empty = "Empty"

class Application(ApplicationLayout):
    def __init__(self):
        super().__init__(minw=1000)
        self.json = None
        self.monsters = None
        self.runes = None

        self.weight_min_max.data = data.T
        self.monster_selector.addItem("Monster")
        self.monster_selector.setDisabled(True)
        for sel in self.runesets_selectors:
            sel.addItems([empty, *sorted(mapping.runes.sets.values())])

        self.button.clicked.connect(self.select_data_file)
        # self.enable_optimizer()


    def gatherSets(self):
        return dict(Counter([sel.value for sel in self.runesets_selectors if sel.value != empty]))

    def gatherWeightMinMax(self) -> DataFrame[WeightMinMax]:
        return self.weight_min_max.data.astype("Int64").T

    def select_data_file(self, *args):
        file_dialog = QFileDialog(self)
        file_dialog.setFileMode(QFileDialog.FileMode.ExistingFile)
        file_dialog.setNameFilter("JSON Files (*.json)")
        file_dialog.setViewMode(QFileDialog.ViewMode.List)
        if file_dialog.exec():
            file_paths = file_dialog.selectedFiles()
            if file_paths:
                file_path = file_paths[0]  # Get the first selected file
                self.json = extract.json(file_path)
                self.enable_optimizer()

    def enable_optimizer(self):
        self.runes = extract.runes(self.json)
        self.monsters = extract.monsters(self.json)
        self.monster_selector.setEnabled(True)
        self.monster_selector.removeItem(0)
        self.monster_selector.addItems(self.monsters.name.dropna().values)
        self.button.setText("Optimize")
        self.button.clicked.disconnect(self.select_data_file)
        self.button.clicked.connect(self.optimize)

    def optimize(self):
        filtered_runes = self.runes 
        if not self.allow_equipped_checkbox.isChecked():
            filtered_runes = transform.filter_equipped(self.runes, self.monsters, self.monster_selector.value)
        solution = optimize(filtered_runes, self.gatherWeightMinMax(), self.gatherSets())
        if solution is not None:
            self.results.data = solution
                

app = Application()
app.exec()

In [7]:
[*app.monsters.index]

[19918998103,
 19938703052,
 19938703405,
 19942289075,
 19967664494,
 20017419492,
 20062742470,
 19925223766,
 19937808616,
 19947263421,
 19952945148,
 19989447618,
 19925011694,
 19928712019,
 19966001616,
 19971593307,
 20030201176,
 19919096051,
 19920273399,
 19921033040,
 19950665931,
 20006412531,
 20041825260,
 19914665257,
 19926054232,
 19933748907,
 19989671100,
 20010833915,
 20042456303,
 19940257990,
 19968725849,
 20056001876,
 20062246075,
 20030841365,
 20035051593,
 20032778744,
 20063076473,
 20063198116,
 20063198196,
 20063694134,
 20063743304,
 20063810808,
 19918232817,
 20063810601,
 20064898361,
 20064897989,
 20064898217,
 20063198153,
 20063487766,
 20063488222,
 20063810512,
 19922048353,
 19925008999,
 19989662601,
 19913982459,
 19940258367,
 19974936570,
 19985346300,
 19921168140,
 19979210768,
 19989665384,
 19940260597,
 19950773439,
 19970022283,
 19940369162,
 19925008998,
 19925243637,
 19970020483,
 19950691819]

In [2]:
app.runes

,monster,slot,set,hp,hp%,atk,atk%,def,def%,spd%,spd,crt,crd,res,acc
id,,,,,,,,,,,,,,,
45968131124,19918998103,1,Blade,0,0,118,13,0,0,0,11,15,0,8,5
46050920366,19918998103,2,Fatal,332,5,0,47,35,0,0,5,15,0,0,0
45890876944,19918998103,3,Blade,139,0,0,0,118,8,0,6,12,18,0,0
45977263962,19918998103,4,Fatal,0,0,18,13,0,5,0,0,16,80,0,4
45725036318,19918998103,5,Fatal,1800,0,0,7,0,9,0,6,16,0,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45993908594,0,5,Intangible,1800,0,0,6,0,6,0,0,18,0,8,0
45983328994,0,6,Intangible,0,0,0,47,0,0,0,5,20,4,5,0
46092618788,0,6,Intangible,0,7,0,11,11,0,0,0,0,7,8,0


In [3]:
app.monsters.name.values

array(['Lapis', 'Abigail', 'Galleon', 'Sigmarus', 'Aaaliyah', 'Theomars',
       'Beth', 'Verdehile', 'Spectra', 'Astar', 'Raoq', 'Sath', 'Mellia',
       'Lushen', 'Teshar', 'Jamire', 'Homunculus - Attack (Wind)', 'Fran',
       'Belladeon', 'Loren', 'Darion', 'Jeanne', 'Shamann', 'Nicki',
       'Kro', 'Veromos', 'Thrain', nan, 'Odin', 'Lulu and Friends',
       'Vigor', 'Tatu', 'Kahli', 'Bernard', 'Vereesa', nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       'Konamiya', 'Suiki', nan, 'Sieq', nan, 'Colleen', nan, 'Shannon',
       'Jade', nan, 'Dona', 'Ahman', 'Gina', 'Clara', 'Liesel', 'Lariel',
       'Susan', 'Manx'], dtype=object)

In [4]:
app.gatherWeightMinMax().T

,hp,hp%,atk,atk%,def,def%,spd%,spd,crt,crd,res,acc
weight,1,1,1,1,1,1,1,1,1,1,1,1
min,<NA>,<NA>,100,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
max,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
